In [ ]:

import nltk.data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
import types
import pandas as pd
import csv

!pip install allennlp==1.0.0 allennlp-models==1.0.0
#from allennlp.predictors.predictor import Predictor



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
#txt='../sec_cleaning/txt/' #path of txt files from SEC-cleaned  for my Anaconda local


In [ ]:
import glob,os
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive')

print(os.getcwd())

os.chdir("/content/gdrive/My Drive/Colab Notebooks/srl/txt")


print(os.getcwd())

if( not os.path.isdir(os.getcwd()+"/output/")):
  os.mkdir(os.getcwd()+"/output/")
  print(os.getcwd()+"/output/")
  print("directory is created")
else:
  print(os.getcwd()+"/output/")
  print("directory already exists")
  
import MyFun,variables 


rid = 0
sentences = {}
filenames=[]

prefixed_files = [filename for filename in glob.glob("*.txt") if filename.startswith("AAL_")] #try for only American Airlines' 10 years. comment out to parse the entire folder.

#read file path

# for all files replace the below line with the other loop below

#for text_path in glob.glob("*.txt"):  # comment to parse the entire txt folder
for text_path in prefixed_files:
    text = Path(text_path).read_text()
    filenames.append(os.path.basename(text_path))
    sentences[rid] = []
    text = text.strip()
    
    text = text.replace("•"," . ")  
    text = text.replace(".",". ")  

    sentences[rid].extend(sent_detector.tokenize(text))
    rid+=1



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content
/content/gdrive/My Drive/Colab Notebooks/sec_cleaning/txt
/content/gdrive/My Drive/Colab Notebooks/sec_cleaning/txt/output/
directory already exists


In [ ]:
import re

causal_verbs = variables.causal_verbs
reverse_causal_verbs = variables.reverse_causal_verbs

# to be used with SRL

In [ ]:

count = 0

for rid in sentences:
    variables.missed_verbs_new = MyFun.get_unique_fileName(os.path.splitext(filenames[rid])[0]+"_"+variables.missed_verbs,os.getcwd()+"/output/")
    Cause_Effect_Pairs_new = MyFun.get_unique_fileName(os.path.splitext(filenames[rid])[0]+"_"+variables.Cause_Effect_Pairs,os.getcwd()+"/output/")

    print("")
    print(variables.missed_verbs_new)
    print(Cause_Effect_Pairs_new)
    print("")

    missed_verbs_csv = open(os.getcwd()+"/output/"+variables.missed_verbs_new,"w") #<===open and write to a csv file  and New file is created.
    csvOutput = open(os.getcwd()+"/output/"+Cause_Effect_Pairs_new,"w")
    
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
    missed_verbs_writer = csv.writer(missed_verbs_csv, quoting = csv.QUOTE_NONNUMERIC)

    csvWriter.writerow(["file name","direction","verb","cause","effect","sent"])
    missed_verbs_writer.writerow(["verb","sentence"])
    missed_verbs_csv.close()


    for sent in sentences[rid]:
        part_list = MyFun.get_verb_left_right(sent)
        
        if(part_list):
          for parts in part_list:
            if parts:
                verb = parts[0]
                if verb.lower() in causal_verbs:
                    count += 1
                    cause = parts[1]
                    effect = parts[2]
                    csvWriter.writerow([filenames[rid],"forward",verb,cause,effect,sent])
                    print("f", end='')
                if verb.lower() in reverse_causal_verbs:
                    count += 1
                    cause = parts[2]
                    effect = parts[1]
                    csvWriter.writerow([filenames[rid],"reverse",verb,cause,effect,sent])
                    print("b", end='')
                    
                if(count%30==0):
                  csvOutput.close()
                  csvOutput = open(os.getcwd()+"/output/"+
                                   Cause_Effect_Pairs_new,"a") # <===open and write to a csv file  and New rows will be appended.
                  csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)


    csvOutput.close()
    missed_verbs_csv.close()

print("end of loop")

csvOutput.close()
missed_verbs_csv.close()




AAL_2_2019_02_25_Missed_verbs.csv
AAL_2_2019_02_25_Cause_Effect_Pairs.csv

.f..ff...f....f................................................fff.....f.........fff....................ff.........f...f.....f.........f..................f..f...........f.......f.f..ff....ff........f............ff.f........f...f.fff.f.....f..ffff.ffffff......ff...f......f..f..f.................fff.ff....f..........f.......ff.......fff.....fff.ff......ff.ff.....f.............f.f......f.................................fff............ff


AAL_3_2018_02_21_Missed_verbs.csv
AAL_3_2018_02_21_Cause_Effect_Pairs.csv

.f..ff...f...f............................f.......ff.....f.....fff....................ff.........f....f......f.........f...................f..f...........f.....f..f..f.